# Setup

In [97]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango pdflatex

Note: you may need to restart the kernel to use updated packages.


In [98]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

Set up the OpenAI Assistant

In [99]:
from openai import OpenAI
client = OpenAI()

#  get the prompt from the file system
with open('prompt.md', 'r') as file:
    # Read the entire content of the file into a single string variable
    prompt = file.read()

# simple retrieve of the Assistant - not necessary because we are doing an update
# my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# update the assistant with the latest prompt
my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
  # model="gpt-3.5-turbo-1106"
)

In [100]:
# create a thread
my_thread = client.beta.threads.create()

import time

def create_run(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    #extra_instructions = """"""
    extra_instructions = """ ONLY return the content as John Gruber's markdown with no delimiters and with no commentary whatsoever."""


    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


## Initial File Upload and Prompt

In [101]:
import time

# Which example to run
choice = 'medical-assistant'
#choice = 'infra-engineer'
#choice = 'customer-success-manager'
#choice = 'real-estate-litigator'

destination_directory = "examples/" + choice + "/"
html_destination = destination_directory + 'playbook.html'
pdf_destination = destination_directory + 'playbook.pdf'
markdown_destination = destination_directory + 'playbook.md'
word_destination = destination_directory + 'playbook.docx'

company_file = "examples/" + choice + "/about-company.md"

# Upload the company description
file_company_values = client.files.create(
  file=open(company_file, "rb"),
  purpose="assistants"
)

job_description_file = "examples/" + choice + "/job-description.md"
# Open the file and read the contents
with open(job_description_file, 'r') as file:
    # Read the entire content of the file into a single string variable
    job_description = file.read()

# Upload the job description file
file_job_description = client.files.create(
  file=open(job_description_file, "rb"),
  purpose="assistants"
)

# give OpenAI time to process the file
time.sleep(10)


# Priming the Pump

In [102]:
# Refresh the context to ensure it has read the files
primer_values = create_run(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the about company file, using the company name.
            """, [file_company_values.id])
print(primer_values)

```markdown
# About Pacific Wellness Group - Academic Healthcare Consortium

## Mission
To be the premier community network for the Pacific Wellness Group System, delivering the best patient experience with the most effective clinical outcomes.

## Vision
Healing humanity through science and compassion, one patient at a time.

## Values
- **Innovation**: Embracing innovation, change, and continuous improvement to raise the quality of care and shape the future of medicine.
- **Connection**: Cultivating community, connection, and care for each other to nurture trust, respect, belonging, and shared purpose.
- **Teamwork**: Collaborating across roles, ranks, units, and departments to unlock possibilities for greater impact and fulfillment.
- **Mentorship**: Nurturing and pursuing ongoing professional growth to enjoy impactful and fulfilling careers.
- **Participation**: Freely sharing opinions and valuing diverse backgrounds and contributions to improve decision-making, outcomes, and engag

In [103]:
# Refresh the context to ensure it has read the files and create context. Not included in final doc.
primer_job = create_run(client,my_thread,my_assistant,"""
            Write the job title contained in the job description file.
            Then, create the list of 5 technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description. Create the list of 5 behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            For each competency, list 3 or 4 subcompetencies and their descriptions to enable more deep exploration.
            """, [file_job_description.id])
print(primer_job)


RuntimeError: Run did not complete successfully.

# Create Introduction

In [ ]:
playbook_title = create_run(client,my_thread,my_assistant,"""
            Generate the title for the playbook as a simple string with no quotation marks
            """)
print(playbook_title)

Interview Playbook for Medical Assistant II at Pacific Wellness Group




In [ ]:
introduction = create_run(client,my_thread,my_assistant,"""
            Generate the introduction for the guide with no headings
            """)
print(introduction)

This Interview Playbook is designed to facilitate the selection of a highly qualified Medical Assistant II at Pacific Wellness Group. The guide outlines the structured approach to evaluating candidates' technical and behavioral competencies aligned with the company's values. The intent is to ensure a thorough and consistent interview process, focusing not only on the required technical skills but also on the candidate's fit within the company culture. As Medical Assistants play a crucial role in patient care and operational efficiency, this playbook assists the hiring team in identifying professionals who will contribute positively to the dynamic and compassionate environment that Pacific Wellness Group is known for.




In [ ]:
interview_structure = create_run(client,my_thread,my_assistant,"""
            Generate the Interview Structure with no headings. Include mention of the competencies in the interview structure.
            """)
print(interview_structure)

The interview for the Medical Assistant II position at Pacific Wellness Group should be structured over approximately 60 to 90 minutes, divided into distinct sections as follows:

- **Introduction (5-10 minutes)**: Begin with a brief welcome and provide an overview of Pacific Wellness Group, the interview flow, and discuss the candidate’s understanding of the Medical Assistant II role.

- **Review of Basic Qualifications (5-10 minutes)**: Quickly verify the candidate's background, certifications, and basic requisites for the job.

- **Technical Assessment (20-30 minutes)**: Dive into the five key technical competencies, which include Clinical Procedures Proficiency, Communication Skills, Health and Safety Compliance, Computer and EMR Proficiency, and Organizational Skills. Ensure to observe the candidate’s articulation of their skills and experience in each area.

- **Behavioral Assessment (20-30 minutes)**: Explore the behavioral competencies crucial to the role and company culture, s

# Create Evaluation Criteria

In [ ]:
eval_criteria = create_run(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

The evaluation of candidates for the Medical Assistant II position will be based on a structured set of criteria that reflect both technical and behavioral competencies. The following job scorecard serves as a record to assess and comment on each candidate's qualifications throughout the interview process.

| Competency                       | Assessment    | Comments by Interviewer                   |
| -------------------------------- | ------------- | ----------------------------------------- |
| Clinical Procedures Proficiency  |                |                                           |
| Communication Skills             |                |                                           |
| Health and Safety Compliance     |                |                                           |
| Computer and EMR Proficiency     |                |                                           |
| Organizational Skills            |                |                            

# Create Technical Assessment

In [ ]:
technical_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

Evaluating technical competencies is critical to ensure that the candidate possesses the practical skills necessary for the Medical Assistant II position at Pacific Wellness Group.

### Technical Competence 1: Clinical Procedures Proficiency

A Medical Assistant II must exhibit a strong capability in performing a wide range of clinical duties. They should be well-versed in patient care techniques, assisting with medical procedures, patient documentation, and equipment handling to ensure efficient and quality care provision.

#### Lead Question

> **"Describe a complex clinical procedure you've assisted with. What role did you play, and how did you ensure it was executed correctly?"**

_Look for detailed explanations to assess the candidate's understanding of clinical procedures and their ability to perform them accurately._

#### Diving Deeper

- **Patient Care**: "Can you walk us through how you prepare a patient for a specific common procedure in your current

In [ ]:
technical_assessment_2 = create_run(client,my_thread,my_assistant,"""
    Generate the 2nd Technical Assessment. Don't include the main section header.
    """)
print(technical_assessment_2)

### Technical Competence 2: Communication Skills

Being able to communicate effectively with patients, their families, and the medical team is vital for a Medical Assistant. This competency ensures that information is conveyed clearly and efficiently, promoting excellent patient care and collaborative teamwork.

#### Lead Question

> **"Describe a time when you had to explain a complex medical procedure or condition to a patient who was anxious or confused. How did you handle it?"**

_Look for clarity, empathy, and the ability to tailor communication style to the patient’s needs._

#### Diving Deeper

- **Verbal Communication**: "Give an example of a difficult conversation you've had with a family member of a patient. What was the outcome?"
- **Written Communication**: "Can you describe your experience with creating patient education materials or providing written instructions for aftercare?"
- **Patient Education**: "How do you ensure that a patient fully understands their treatment p

In [ ]:
technical_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Health and Safety Compliance

A Medical Assistant II must maintain a strong adherence to health and safety protocols to ensure the well-being of patients and colleagues. This includes staying updated with the latest healthcare regulations and safety procedures.

#### Lead Question

> **"Can you share an experience where you identified a breach in health or safety protocols? What actions did you take to address it?"**

_Look for proactive identification of issues, knowledge of protocols, and appropriate action taken._

#### Diving Deeper

- **BLS Certification**: "How have you applied your BLS skills in a real-world setting, and what was the outcome of that situation?"
- **Biohazard Handling**: "What procedures do you follow when handling biohazards to ensure safety for yourself and others?"
- **Infection Control**: "Explain how you implement infection control guidelines in your daily duties and during an outbreak or pandemic."
- **Regulatory Awareness**: "Di

In [ ]:
technical_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Computer and EMR Proficiency

The role demands proficient use of electronic medical records systems and computers to maintain efficient operations and accurate patient records, in line with Pacific Wellness Group's emphasis on innovation and continuous improvement.

#### Lead Question

> **"Discuss your experience with electronic medical records. How do you ensure data integrity and security when entering and retrieving patient information?"**

_Look for examples demonstrating attention to detail, organization, and a commitment to patient privacy and data accuracy._

#### Diving Deeper

- **EMR Management**: "What electronic medical record systems are you familiar with, and what features do you find most beneficial for clinical workflow?"
- **Microsoft Office Skills**: "Provide an example of how you have used Microsoft Office tools to improve a clinical process or patient communication."
- **Digital Literacy**: "Tell us about a time when you had to learn a n

In [ ]:
technical_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Organizational Skills

Strong organizational skills are critical for Medical Assistant IIs to effectively manage their varied responsibilities within the clinic, ensuring the smooth operation of patient care and support of healthcare services.

#### Lead Question

> **"Tell us about a time when you had to juggle multiple tasks at once in a clinical setting. How did you prioritize and manage these tasks to maintain quality patient care?"**

_Look for evidence of efficient prioritization, multitasking ability, and effectiveness in handling a fast-paced environment._

#### Diving Deeper

- **Time Management**: "How do you typically organize your day to accommodate unexpected tasks or emergencies while still completing scheduled duties?"
- **Resource Coordination**: "Describe a situation where you had to manage clinic resources during a busy day. What approach did you take?"
- **Work Prioritization**: "When faced with simultaneous urgent tasks, how do you decide

In [ ]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [ ]:
behavioral_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

The behavioral assessment is intended to delve into how the candidate's values align with Pacific Wellness Group's culture, focusing on qualities essential for the Medical Assistant II role. 

### Behavioral Competence 1: Adherence to Service Excellence (C-I-CARE)

Candidates must demonstrate a strong commitment to providing exceptional service and care, in line with Pacific Wellness Group's C-I-CARE framework which emphasizes compassion, integrity, accountability, respect, and engagement.

#### Lead Question

> **"Reflecting on the C-I-CARE framework, can you provide an example of how you've embodied these values in your past work experience?"**

_Look for specific instances where the candidate has gone above and beyond to provide exemplary care and service._

#### Diving Deeper

- **Empathy**: "Describe a time when you empathized with a patient or their family and how it impacted the care provided."
- **Accountability**: "Share an occasion where you made a m

In [ ]:
behavioral_assessment_2 = create_run(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Teamwork and Collaboration

Effective teamwork and the ability to collaborate across different roles and departments are fundamental to the success of Pacific Wellness Group's mission. Candidates should possess strong interpersonal skills and a team-oriented mindset, reflecting the value of unified effort towards a common goal.

#### Lead Question

> **"Can you tell us about a time when you had to collaborate with a diverse team to achieve a healthcare outcome? How did you ensure that everyone's expertise was utilized?"**

_Look for insights into the candidate's capacity to work collaboratively, promote team cohesion, and integrate diverse perspectives._

#### Diving Deeper

- **Interdisciplinary Coordination**: "Describe an interdisciplinary project you've been a part of. What role did you play and what was the outcome?"
- **Contribution to Team Goals**: "What method do you typically use to contribute to team goals and objectives in a clinical environment?

In [ ]:
behavioral_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Continuous Improvement (Innovation)

Aligning with Pacific Wellness Group's value of embracing innovation, a Medical Assistant II should display a commitment to continuous learning and improvement. Candidates should demonstrate how they seek to enhance their skills and processes to contribute to advancements in patient care and operational excellence.

#### Lead Question

> **"Pacific Wellness Group values continuous improvement. Can you discuss an example of how you have identified and implemented a change that improved a process or outcome in your work?"**

_Look for a proactive approach to problem-solving, willingness to initiate change, and an innovative mindset._

#### Diving Deeper

- **Process Optimization**: "Describe a time when you suggested a new approach to a routine task that led to improvements in efficiency or outcomes."
- **Learning and Development**: "How do you stay current with the latest developments in medical assisting and apply them t

In [ ]:
behavioral_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Professional Growth (Mentorship)

The Medical Assistant II at Pacific Wellness Group should embody the value of mentorship by actively seeking and fostering professional development. They should contribute to creating an environment that supports growth and knowledge sharing among colleagues.

#### Lead Question

> **"Mentorship is a key value at Pacific Wellness Group. Can you provide an example of how you have either mentored someone else or benefited from mentorship in your professional journey?"**

_Look for examples reflecting commitment to personal growth and the development of others, as well as the ability to provide or receive guidance._

#### Diving Deeper

- **Skill Enhancement**: "How do you approach continuous skill improvement in your career as a Medical Assistant?"
- **Knowledge Sharing**: "Describe a situation where you shared your expertise with colleagues or patients to improve their understanding or skills."
- **Mentoring Others**: "Have 

In [ ]:
behavioral_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Community Building (Connection)

Pacific Wellness Group emphasizes the importance of fostering connections and building a supportive community. Candidates are expected to contribute to a culture where everyone feels valued, supported, and part of a cohesive group.

#### Lead Question

> **"Building a supportive community is vital at Pacific Wellness Group. Can you share an experience where you played a key role in building or enhancing a sense of community in your workplace?"**

_Look for instances indicating the candidate's ability to create bonds, foster inclusiveness, and build a sense of belonging._

#### Diving Deeper

- **Relationship Building**: "What strategies do you use to develop and maintain positive working relationships with coworkers from different departments?"
- **Inclusivity**: "Give us an example of how you have made others feel included in a team or project, especially those who might otherwise feel marginalized."
- **Community Participa

In [ ]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [ ]:
practical_assignment = create_run(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment for the Medical Assistant II candidate at Pacific Wellness Group will involve simulating a typical clinical task. This could be a mock patient consultation where the candidate is required to take a patient history, explain a common procedure, prepare the room, and demonstrate how they would document the visit in the EMR system.

Objectives:
- Assess the candidate's ability to apply their clinical skills in a practical setting.
- Evaluate the candidate's proficiency with EMR and documentation.
- Observe the candidate's communication skills and bedside manner with patients.

Evaluation Criteria:
- Accuracy in obtaining patient information.
- Clarity and empathy in communication with the "patient."
- Effectiveness in preparing for and explaining the procedure.
- Proficiency in documenting the consultation in the EMR.

Interviewers should assess the candidate's approach, noting their organization, attention to detail, and ability to provide

# Create Team Interaction

In [ ]:
team_interaction = create_run(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

For the Medical Assistant II position at Pacific Wellness Group, it is essential to assess how well the candidate will mesh with the existing team. The team interaction component of the interview provides insights into the candidate's interpersonal skills, adaptability, and ability to work collaboratively with colleagues.

During this stage, the candidate may be introduced to potential peers and supervisors. This could include a brief meet and greet, a shadowing period, or a more structured group discussion with team members on relevant topics.

Rationale for team involvement:
- To observe the candidate's interaction with potential colleagues.
- To garner feedback from staff who will be working directly with the new hire.
- To determine the candidate’s fit within the team dynamic and company culture.

Potential discussion points or activities:
- How the candidate approaches teamwork and handles joint responsibilities.
- Feedback on the candidate's communication style and r

# Create Candidate Q&A

In [ ]:
candidate_questions = create_run(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

It's critical for candidates to have the opportunity to ask questions about their potential new role and organization. This can provide valuable insights into their priorities and interest in the position at Pacific Wellness Group.

Here are six questions candidates might ask, along with comprehensive answers:

1. **"What opportunities for professional development does Pacific Wellness Group offer for Medical Assistant IIs?"**
   _Answer: Pacific Wellness Group values continuous learning and offers various professional development programs, including in-house training, mentorship opportunities, access to medical seminars, and support for ongoing education relevant to the medical field._

2. **"How does the clinic measure success for someone in this position?"**
   _Answer: Success is measured by a combination of patient satisfaction scores, adherence to health and safety protocols, efficiency in clinical operations, and the ability to work effectively with the healthc

# Finalize the Playbook HTML

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [111]:
import markdown

def convert_to_html(text):
    return markdown.markdown(text, extensions=['extra','mdx_truly_sane_lists'])

html_job_description = convert_to_html(job_description)
html_playbook_title = convert_to_html(playbook_title)
html_intro = convert_to_html(introduction)
html_interview_structure = convert_to_html(interview_structure)
html_technical_assessment = convert_to_html(technical_assessment)
html_behavioral_assessment = convert_to_html(behavioral_assessment)
html_practical_assignment = convert_to_html(practical_assignment)
html_candidate_questions = convert_to_html(candidate_questions)
html_team_interaction = convert_to_html(team_interaction)
html_eval_criteria = convert_to_html(eval_criteria)

html_technical_assessment_1 = convert_to_html(technical_assessment_1)
html_technical_assessment_2 = convert_to_html(technical_assessment_2)
html_technical_assessment_3 = convert_to_html(technical_assessment_3)
html_technical_assessment_4 = convert_to_html(technical_assessment_4)
html_technical_assessment_5 = convert_to_html(technical_assessment_5)

html_behavioral_assessment_1 = convert_to_html(behavioral_assessment_1)
html_behavioral_assessment_2 = convert_to_html(behavioral_assessment_2)
html_behavioral_assessment_3 = convert_to_html(behavioral_assessment_3)
html_behavioral_assessment_4 = convert_to_html(behavioral_assessment_4)
html_behavioral_assessment_5 = convert_to_html(behavioral_assessment_5)



## Build the HTML

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [112]:
# Open the file and read the contents
with open('templates/styles.css', 'r') as file:
    # Read the entire content of the file into a single string variable
    styles = file.read()


html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")


# Open the HTML file and read its content
with open('templates/template.html', 'r') as file:
    html_template = file.read()

# Replace the placeholders with the actual variables
html_content = html_template.format(
    styles=styles,
    html_playbook_title=html_playbook_title,
    publication_date=publication_date,
    html_intro=html_intro,
    html_interview_structure=html_interview_structure,
    html_job_description=html_job_description,
    html_eval_criteria=html_eval_criteria,
    html_technical_assessment_1=html_technical_assessment_1,
    html_short_notes=html_short_notes,
    html_technical_assessment_2=html_technical_assessment_2,
    html_notes=html_notes,
    html_technical_assessment_3=html_technical_assessment_3,
    html_technical_assessment_4=html_technical_assessment_4,
    html_technical_assessment_5=html_technical_assessment_5,
    html_behavioral_assessment_1=html_behavioral_assessment_1,
    html_behavioral_assessment_2=html_behavioral_assessment_2,
    html_behavioral_assessment_3=html_behavioral_assessment_3,
    html_behavioral_assessment_4=html_behavioral_assessment_4,
    html_behavioral_assessment_5=html_behavioral_assessment_5,
    html_practical_assignment=html_practical_assignment,
    html_team_interaction=html_team_interaction,
    html_candidate_questions=html_candidate_questions
)

# Write HTML string to a file
with open(html_destination, 'w') as file:
    file.write(html_content)


# Display the Markdown-formatted text in the notebook
# from IPython.display import display, HTML
# display(HTML(html_content))


# Create the Documents

### Create PDF from HTML

In [113]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['templates/styles.css'], full_fonts=True, hinting=True, presentational_hints=True)



## Create a markdown version

For use in GitHub as an example

In [114]:
notes_snippet_md = """##### Notes\n
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________\n
Rating (1-5): \\__________\n\n\n"""

from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

playbook_body_md = f"""
## Interview Structure
{interview_structure}

## Job Description
{job_description}

{eval_criteria}

{technical_assessment_1}
{notes_snippet_md}
            
{technical_assessment_2}
{notes_snippet_md}

{technical_assessment_3}
{notes_snippet_md}

{technical_assessment_4}
{notes_snippet_md}
            
{technical_assessment_5}
{notes_snippet_md}
            
{behavioral_assessment_1}
{notes_snippet_md}

{behavioral_assessment_2}
{notes_snippet_md}
            
{behavioral_assessment_3}
{notes_snippet_md}
            
{behavioral_assessment_4}
{notes_snippet_md}
            
{behavioral_assessment_5}
{notes_snippet_md}
            
{practical_assignment}
{notes_snippet_md}

{team_interaction}
{notes_snippet_md}

{candidate_questions}
"""

print(len(playbook_body_md))

# exec_summary = create_run(client,my_thread,my_assistant,f"""
#    Write a summary for this document:
#
#    {playbook_body_md}
#    """)

exec_summary = create_run(client,my_thread,my_assistant,f"""
    Write a detailed introduction for the document. Be clear and concise and don't use flowery language.
    """)

    
print(exec_summary)

January 05, 2024 1:32 PM
35630
```markdown
# Interview Playbook for Medical Assistant II

## Introduction

This document serves as a comprehensive guide for the hiring team at Pacific Wellness Group in the selection of candidates for the position of Medical Assistant II. It is tailored to assess both technical and behavioral competencies critical for the role, in alignment with the company's commitment to quality patient care and innovation in healthcare. The interview process outlined here is designed to objectively evaluate each candidate's skills, work ethic, and alignment with our core values, thereby ensuring a thorough and fair hiring process.
```




In [115]:
playbook_md = f"""--- 
title: {playbook_title}
author: Created by [Deliverables.ai](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

## Introduction
{exec_summary}
            
{playbook_body_md}
"""


# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(playbook_md)


### Create Word Document from Markdown

In [116]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=2 --shift-heading-level-by -1
# !pandoc -s {markdown_destination} -o {html_destination} --template='templates/html-reference.html --toc=True
# !pandoc --pdf-engine "weasyprint" -s {html_destination} -o {pdf_destination}

# use this extension to shift headings to differ between HTML and docx
# --shift-heading-level-by -1

# Convert from markdown to PDF via HTML
# !pandoc --pdf-engine "weasyprint" -s {markdown_destination} -o {pdf_destination} --reference-doc="custom-reference.docx" --toc=True
# !pandoc --pdf-engine "weasyprint" -s {word_destination} -o {pdf_destination}


In [117]:
pptx_destination = destination_directory + "playbook.pptx"
!pandoc -s {markdown_destination} -o {pptx_destination}